# Projet maison n° 3

In [1]:
# imports
import numpy as np
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
import os
# names us
def df_names_us():
    df_list = [pd.read_csv("./names/"+x, names = ["name","gender","births"]).assign(year=int(x.split(".")[0][3:])) for x in os.listdir("./names/")[1:]]
    df = pd.concat(df_list, axis=0, ignore_index=True)[["year","name","gender","births"]]
    return df

df_us = df_names_us()
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [20]:
# names fr
def df_names_fr():
    df = pd.read_csv("nat2020_csv/nat2020.csv", sep=";")#, header = 0)
    df.columns = ["gender","name","year","births"]
    df = df[["year","name","gender","births"]]
    df = df[(df["year"] != "XXXX") & (df["name"] != "_PRENOMS_RARES")]
    df = df.astype(df_us.dtypes.to_dict())
    df["name"] = df["name"].apply(str)
    df = df[df["name"].apply(len)>=2]
    df["name"] = df["name"].str.capitalize()
    df["gender"] = df["gender"].apply(lambda x: "F" if int(x) == 2 else "M")
    df = df.sort_values(["year","gender","births","name"],ascending=[True,True,False,True])
    df = df.reset_index(drop=True)
    return df

df_fr = df_names_fr()
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [21]:
# taux de change
def df_taux_change(devises=None):
    df = pd.read_csv("Webstat_Export_20210929.csv", sep=";", index_col=0, skiprows=range(1,6))
    df.columns = pd.Series(df.columns).apply(lambda x: x[-4:-1]).tolist()
    df.index.name = None
    df.index = pd.to_datetime(df.index)
    df = df.dropna(how="all",axis=1)
    df = df.replace("-",np.nan)
    df = df.dropna(axis=0)
    df = df.apply(lambda x: x.str.replace(",","."), axis = 0)
    if devises:
        df = df[devises]
    df = df.apply(lambda x: pd.to_numeric(x),axis=0)
    return df

df = df_taux_change()
df.head(10)

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,RON,RUB,SEK,SGD,SIT,SKK,THB,TRY,USD,ZAR
2006-12-29,1.6691,1.9558,2.8141,1.5281,1.6069,10.2793,0.5782,27.485,7.4560,15.6466,...,3.3835,34.680,9.0404,2.0202,239.64,34.435,46.770,1.8640,1.3170,9.2124
2006-12-28,1.6699,1.9558,2.8175,1.5268,1.6058,10.2935,0.5782,27.540,7.4573,15.6466,...,3.3980,34.694,9.0463,2.0209,239.64,34.561,47.252,1.8648,1.3173,9.2590
2006-12-27,1.6765,1.9558,2.8343,1.5297,1.6058,10.2923,0.5782,27.590,7.4543,15.6466,...,3.3695,34.666,9.0230,2.0217,239.65,34.349,47.776,1.8751,1.3159,9.2198
2006-12-22,1.6792,1.9558,2.8391,1.5240,1.6029,10.3113,0.5781,27.573,7.4536,15.6466,...,3.3510,34.703,8.9888,2.0310,239.66,34.378,48.019,1.8810,1.3192,9.2195
2006-12-21,1.6785,1.9558,2.8460,1.5155,1.6032,10.3012,0.5781,27.525,7.4545,15.6466,...,3.3759,34.673,8.9661,2.0303,239.69,34.585,48.159,1.8836,1.3178,9.2280
2006-12-20,1.6820,1.9558,2.8341,1.5137,1.6039,10.3184,0.5781,27.418,7.4529,15.6466,...,3.4029,34.717,9.0038,2.0316,239.66,34.688,47.400,1.8855,1.3203,9.2441
2006-12-19,1.6824,1.9558,2.8200,1.5217,1.6024,10.2911,0.5781,27.745,7.4538,15.6466,...,3.4169,34.641,9.0252,2.0326,239.64,34.880,47.073,1.8845,1.3158,9.2168
2006-12-18,1.6779,1.9558,2.8101,1.5114,1.5993,10.2406,0.5781,27.700,7.4537,15.6466,...,3.4185,34.566,9.0555,2.0230,239.66,34.839,47.022,1.8663,1.3095,9.1727
2006-12-15,1.6803,1.9558,2.8336,1.5151,1.5977,10.2587,0.5781,27.755,7.4545,15.6466,...,3.4264,34.584,9.0633,2.0218,239.66,34.853,46.253,1.8676,1.3106,9.2368
2006-12-14,1.6848,1.9558,2.8461,1.5242,1.5987,10.3142,0.5781,27.843,7.4544,15.6466,...,3.4293,34.690,9.0602,2.0322,239.65,34.866,46.515,1.8790,1.3192,9.2604


### Tests

In [22]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [23]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [24]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.259s

OK
